In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 1. 数据预处理

In [2]:
ratings_df = pd.read_csv('ratingsProcessed.csv')

In [3]:
train_rating_df = pd.read_csv('train_rating.csv')

In [4]:
test_rating_df = pd.read_csv('test_rating.csv')

In [5]:
userNo,movieNo = ratings_df['userId'].max()+1,ratings_df['movieId'].max()+1

In [6]:
testNo = np.shape(test_rating_df)[0]

   # 创建训练集的用户评分矩阵train_rating和测试集用户评分矩阵test_rating

In [7]:
train_rating = np.zeros((userNo,movieNo))

In [8]:
for index ,row in train_rating_df.iterrows():
    train_rating[int(row['userId']),int(row['movieId'])] = row['rating']

In [9]:
test_rating = np.zeros((userNo,movieNo))

In [10]:
for index ,row in test_rating_df.iterrows():
    test_rating[int(row['userId']),int(row['movieId'])] = row['rating']

# 创建训练集的用户记录矩阵train_record和测试集用户记录矩阵test_record

In [11]:
train_record = train_rating>0

In [12]:
train_record = np.array(train_record,dtype=int)

In [13]:
test_record = test_rating>0

In [14]:
test_record = np.array(test_record,dtype=int)

# 2. 构建Latent Factor Model

In [15]:
f = 10

In [16]:
def CreateLFM():
    LFM_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_Q = tf.Variable(tf.random_normal([movieNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_predict_rating = tf.matmul(LFM_P,LFM_Q,transpose_b=True)
    LFM_loss = tf.reduce_sum(((train_rating-LFM_predict_rating)*train_record)**2) + lamda*((tf.reduce_sum(LFM_P)**2)+(tf.reduce_sum(LFM_Q)**2))
    return LFM_loss,LFM_predict_rating

In [17]:
#P = tf.Variable(tf.random_normal([userNo,f],stddev=0.35,dtype=tf.float32))
#P = tf.Variable(tf.random_uniform([userNo,f],minval=0.0,maxval=1.0,dtype=tf.float32))

In [18]:
#Q = tf.Variable(tf.random_normal([movieNo,f],stddev=0.35,dtype=tf.float32))
#Q = tf.Variable(tf.random_uniform([movieNo,f],minval=0.0,maxval=1.0,dtype=tf.float32))

In [19]:
lamda = 0.01

In [20]:
#step = tf.Variable(0, trainable=False) #学习步数，不可训练

In [21]:
learning_rate = 1e-3
#learning_rate = tf.train.exponential_decay(1e-3, step, 1, 0.9, staircase = True)

In [22]:
#loss = tf.reduce_sum(((train_rating-tf.matmul(P,Q,transpose_b=True))*train_record)**2) + lamda*((tf.reduce_sum(P)**2)+(tf.reduce_sum(Q)**2))

# 3.选择优化器（Adam or GD ?)

In [23]:
#optimizer = tf.train.AdamOptimizer(learning_rate)
optimizer = tf.train.AdamOptimizer()
#optimizer = tf.train.GradientDescentOptimizer(1e-5)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [24]:
loss,predict = CreateLFM()
#train = optimizer.minimize(loss,global_step= step)
train = optimizer.minimize(loss)

# 4.训练模型

In [25]:
RMSE = (tf.reduce_sum(((test_rating-predict)*test_record)**2)/testNo)**0.5

In [26]:
tf.summary.scalar("RMSE",RMSE)

<tf.Tensor 'RMSE:0' shape=() dtype=string>

In [27]:
summaryMerged = tf.summary.merge_all()

In [28]:
filename='tensorboard'

In [29]:
writer = tf.summary.FileWriter(filename)

In [30]:
sess = tf.Session()

In [31]:
init = tf.global_variables_initializer()

In [32]:
sess.run(init)

In [33]:
for i in range(1000): 
    _, movie_summary = sess.run([train,summaryMerged])
    writer.add_summary(movie_summary,i)